# Projet Multimédia: Segementation audio

### Becko Camara; Guobao LI; Rongbo Liu; Shiting LI

But: 
      Reconnaissance de locuteurs dans une bande audio et détection de changement de locuteurs.


Données: 
               Bande audio

Outils:
          1. pyAudioAnalysis
          2. Jupyter

# 1. Segmentation

## 1.1. Concepiton

- Déterminer la taille de la fênetre et la taille du bloc.

- Extraction des caractéristiques 
On utilise readAudioFile() de la bibliothèque pour lire le fichier audio, puis obtenir sa fréquence d'échantillonage et le vecteur qui lui correspond. Ensuite, avec la méthode stFeatuureExtraction(), on extrait les caractéristiques de la bande audio sur des fenêtres de 50 ms qu'on incrémentées de 25 ms

- Calcul des MFCCs 
MFCC pour Mel Frequency Cepstrum Coefficient, sont des coefficients basés sur les caractéristiques vocales humaines. il y a une correspondance non linéaire entre féquence mel et fréquence Hz. Mel Frequency Cepstral Coefficients (MFCC) est l'utilisation d'une telle relation entre eux, calculer les caractéristiques des spectrales HZ. Nous avons donc utilisé MFCC pour la reconnaissance vocale.

- Calculer la distribution de gauss de deux blocs consécutifs 

- On combine ces deux blocs, et on calcule la distribution de gauss du bloc combiné

- On stocke le résultat dans le tableaux

## 1.2. Realisation

Voici le réalisation de  code python:

In [1]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import numpy as np
import math
import matplotlib.pyplot as plt
import time
from numpy import *

Import des dépendances et les bibliothèques néssaires:

- audioBasicIO

- audioFeatureExtraction

- numpy(stockage et manutention des grandes matrices)

- math

- matplotlib.pyplot

In [2]:
# main process
[Fs, x] = audioBasicIO.readAudioFile("data/diarizationExample.wav")

/usr/lib/python2.7/dist-packages/scipy/io/wavfile.py:42: WavFileWarning: Unknown wave file format
  warnings.warn("Unknown wave file format", WavFileWarning)


- audioBasicIO.readAudioFile(path)
Cette fonction retourne un tableau numpy qui stocke l'échantillon audio d'un WAV.

*Il y a une warnning ici car le format de wav.*

In [3]:
TIME_OF_WINDOW = 0.050	#a window = 0.05s
TIME_OF_STEP = 0.025		#step = 0.01s
SIZE_OF_WINDOW = int(TIME_OF_WINDOW * Fs)	#the number of frame for one window
SIZE_OF_STEP = int(TIME_OF_STEP * Fs)		#the number of frame for one step
BLOCK_SIZE = 4		#a block has (6 * SIZE_OF_STEP) frame
BLOCK_STEP = 2

- Determiner le fênetre et le bloc

- Une taille de fenêtre de 50 ms et une étape de 25 ms.

In [4]:
# variables 
END_OF_FILE = 0
FIRST_PAIR = 1
INDEX_BOUCLE = 1

In [5]:
def getMFCCs(block_start, block_end):
	return attribute[8:20,block_start:block_end+1]

def getMFCCsFromTime(moment_start, moment_end):
	block_start = int(moment_start / BLOCK_STEP / TIME_OF_STEP - 1)
	block_end = int(moment_end / BLOCK_STEP / TIME_OF_STEP - 1)
	return getMFCCs(block_start, block_end)

- Calculer les MFCCs.

In [6]:
def gauss(x, mean, cov):
	[n, d] = x.shape
	[j, k] = cov.shape
	if (j != n) | (k != n):
		raise Exception("Dimension of the covariance matrix and data should match")
	invcov = cov.T
	mean = np.reshape(mean, (1, n))

	x = x - (np.ones((d, 1))*mean).T
	fact = np.sum(((np.dot(invcov, x))*x), axis = 1)

	y = np.exp(-0.5*fact)

	y = np.divide(y, math.pow((2*math.pi), n)*np.std(cov))

	return y

- Définir la méthode de Gauss pour calculer la distribution de gauss des deux blocs consécutifs

In [7]:
# feature extraction from the library pyAudioAnalysis
attribute = audioFeatureExtraction.stFeatureExtraction(x, Fs, SIZE_OF_WINDOW, SIZE_OF_STEP)


- Feature extraction

- stFeatureExtraction()
pour extraire les courtes terme feature séquencesde pour un signal audio, en utilisant une taille de frame de 50 ms et une étape de frame de 25 ms.

In [8]:
# relationship between the similarity and the timestamp in the audio
relation = [[1 for col in range(2)] for row in range(attribute.shape[1]/BLOCK_STEP)]

In [9]:
while (END_OF_FILE == 0):
	if (FIRST_PAIR == 1):
		# for the first pari of the block
		block_i_index_start = 0
		block_i_index_end = BLOCK_SIZE
		block_i_attribute = getMFCCs(block_i_index_start, block_i_index_end)


		block_i_mean = np.mean(block_i_attribute, axis=1)
		block_i_cov = np.cov(block_i_attribute)
		block_i_log_like = np.log(gauss(block_i_attribute, mean=block_i_mean, cov=block_i_cov))



		block_j_index_start = block_i_index_end + 1
		block_j_index_end = block_j_index_start + BLOCK_SIZE - 1
		block_j_attribute = getMFCCs(block_j_index_start, block_j_index_end)


		block_j_mean = np.mean(block_j_attribute, axis=1)
		block_j_cov = np.cov(block_j_attribute)
		block_j_log_like = np.log(gauss(block_j_attribute, mean=block_j_mean, cov=block_j_cov))


		FIRST_PAIR = 0
        
        else:
		#for the rest of the block
		block_j_index_start += BLOCK_STEP
		block_j_index_end += BLOCK_STEP

		new_attribute = getMFCCs(block_j_index_end-BLOCK_STEP+1, block_j_index_end)

		#the following code is for the object that to avoid recalculate the overlap between the block after moved and before moved
		block_i_index_start += BLOCK_STEP
		block_i_index_end += BLOCK_STEP
		block_i_attribute[:,0:block_i_attribute.shape[1]-new_attribute.shape[1]] = block_i_attribute[:,new_attribute.shape[1]:block_i_attribute.shape[1]]
		block_i_attribute[:,block_i_attribute.shape[1]-new_attribute.shape[1]:block_i_attribute.shape[1]] = block_j_attribute[:,0:new_attribute.shape[1]]

		block_i_mean = np.mean(block_i_attribute, axis=1)
		block_i_cov = np.cov(block_i_attribute)
		block_i_log_like = np.log(gauss(block_i_attribute, mean=block_i_mean, cov=block_i_cov))


		block_j_attribute[:,0:block_j_attribute.shape[1]-new_attribute.shape[1]] = block_j_attribute[:,new_attribute.shape[1]:block_j_attribute.shape[1]]
		block_j_attribute[:,block_j_attribute.shape[1]-new_attribute.shape[1]:block_j_attribute.shape[1]] = new_attribute[:,0:new_attribute.shape[1]]

		block_j_mean = np.mean(block_j_attribute, axis=1)
		block_j_cov = np.cov(block_j_attribute)
		block_j_log_like = np.log(gauss(block_j_attribute, mean=block_j_mean, cov=block_j_cov))


	block_union_index_start = block_i_index_start
	block_union_index_end = block_j_index_end
	block_union_attribute = np.concatenate((block_i_attribute, block_j_attribute), axis = 1)
	block_union_mean = np.mean(block_union_attribute, axis=1)
	block_union_cov = np.cov(block_union_attribute)
	block_union_log_like = np.log(gauss(block_union_attribute, mean=block_union_mean, cov=block_union_cov))

	relation[INDEX_BOUCLE-1][0] = np.sum(block_i_log_like) + np.sum(block_j_log_like) - np.sum(block_union_log_like)
	relation[INDEX_BOUCLE-1][1] = (block_i_index_end + block_j_index_start) / 2 * TIME_OF_STEP


	INDEX_BOUCLE += 1

	if block_j_index_end + BLOCK_STEP > attribute.shape[1]:
		END_OF_FILE = 1

- Processus de calcul pour la première partie d'un bloc.

- On calcule les MFCCs de block_i, ensuite on calcule la valeur moyenne des MFCCs, la valeur de covariance des MFCCs, la valeur de Gausse entre les MFCCs, la valeur moyenne des MFCCs et la valeur de covariance des MFCCs.

- Pour le block_j, on fait les mêmes calculs comme block_i.

- On continue à faire les calculs pour la restante partie de bloc.

- Nous avons constamment procédé alternatif pour éviter les calculs répétitifs.

In [10]:
# cut the audio
relation_cut = filter(lambda t: t[0] > 0, relation)

print "=========The data after cutting==========="
for item in relation_cut:
	print item



# print getMFCCsFromTime(0.4, 4.0).shape

=========The data after cutting===========
[193.04016786970868, 0.1]
[322.80179095213964, 0.15000000000000002]
[48.618235376979669, 3.75]
[206.06284495930311, 3.8000000000000003]
[264.46484990545093, 5.800000000000001]
[399.62976041628497, 6.7]
[185.27945423237873, 6.75]
[77.58484185258385, 11.05]
[55.7106174875737, 12.700000000000001]
[86.606563878601378, 13.100000000000001]
[65.574334156685495, 16.6]
[25.698883507655694, 18.75]
[119.17666505648083, 18.85]
[157.12783806256266, 18.900000000000002]
[82.460244094641666, 24.5]
[10.571385024471169, 25.150000000000002]
[1.5434625659503354, 25.25]
[211.66294698210436, 34.2]
[70.795005105859445, 34.35]
[100.16974802846812, 34.4]
[1, 1]
[1, 1]
[1, 1]


- Couper l'audio et délivre le résultat.

- Si la valeur de Gauss est négative, c'est-à-dire que ce clip d'audio est la même personne comme le fragment précédent. Et si ls valeur de Gauss est positive, c'est-à-dire que ce clip d'audio n'est pas la même personne comme le fragment précédent, on va faire une coupe ici.

- Nous ignorons tous les valeurs de Gauss négatives, seule interceptons les valeurs de Gauss positives.

- Pour le resultat [x, y], x est le résultat de la mise en oeuvre de Gauss, y est le point de temps de couper l'audio.

# 2. Reconnaissance

## 2.1 Kmeans

- Le conception de faire une reconnaissance est utiliser k-means pour mettre les block similaire ensemble.
- Le premier fonction **euclDistance(vector1, vector2)** est calculer la distance entre deux vector.
- Le deuxième foction **initCentroids(dataSet, k)** est obtenir k centres sur le data. **k** est le nombre de class. Dans ce projet, le nombre de class est le nombre des locuteurs.
- Le troisième fonction **kmeans(dataSet, k)** est utiliser le distance Euclidean pour mettre les points ensemble. 

In [11]:
#===========Kmeans Definition=====================
#calculate Euclidean distance
def euclDistance(vector1, vector2):
	return sqrt(sum(power(vector2 - vector1, 2)))

# init centroids with random samples
def initCentroids(dataSet, k):
	numSamples, dim = dataSet.shape
	centroids = zeros((k, dim))
	for i in range(k):
		index = int(random.uniform(0, numSamples))
		centroids[i, :] = dataSet[index, :]
	return centroids

# k-means cluster
def kmeans(dataSet, k):
	numSamples = dataSet.shape[0]
	# first column stores which cluster this sample belongs to,
	# second column stores the error between this sample and its centroid
	clusterAssment = mat(zeros((numSamples, 2)))
	clusterChanged = True

	## step 1: init centroids
	centroids = initCentroids(dataSet, k)

	while clusterChanged:
		clusterChanged = False
		## for each sample
		for i in xrange(numSamples):
			minDist  = 100000.0
			minIndex = 0
			## for each centroid
			## step 2: find the centroid who is closest
			for j in range(k):
				distance = euclDistance(centroids[j, :], dataSet[i, :])
				if distance < minDist:
					minDist  = distance
					minIndex = j

			## step 3: update its cluster
			if clusterAssment[i, 0] != minIndex:
				clusterChanged = True
				clusterAssment[i, :] = minIndex, minDist**2

		## step 4: update centroids
		for j in range(k):
			pointsInCluster = dataSet[nonzero(clusterAssment[:, 0].A == j)[0]]
			centroids[j, :] = mean(pointsInCluster, axis = 0)

	print 'cluster complete!'
	return centroids, clusterAssment

def abs (input):
	if input <0 : return - input
	else : return input

### 2.2 Classification


- Dans cette étap, on d'abord créer une novelle liste *temprelation* qui prendre le premier colonne de relation_cut et ajouter un deuxième colonne qui a des valeurs égal 1. 

- Ensuite, on transmettre cette liste à une matrice par utilise ```dataSet= mat(dataSet)```.

- Enfin, on utlise cette matrice pour calculer les centres.

In [12]:
print '======='

temprelation = []
for i in range(len(relation_cut)-3): #-3 because the last 3 numbers are meaningless
    temprelation.append([relation_cut[i][0],1])

#temprelation is a list which takes only first colum of relation_cut and add another colum which values are all 1
#This action alow to transefer the MFCC to a 2-dimension data which use for cluster


## step 1: load data
print "step 1: load data..."
dataSet = temprelation


# step 2: clustering...
print "step 2: clustering..."
dataSet = mat(dataSet)
k = 4
centroids, clusterAssment = kmeans(dataSet, k)
classification =[]
for indexi in range(k):
	classification .append(centroids[indexi][0])

step 1: load data...
step 2: clustering...
cluster complete!


- Pour chaque block, on calculer le distance avec le centre et le point, et marker different locuteur. 

In [13]:
## step 3: Mark speaker to each block
relation_cut_regonize = [] #This list is to store the information with different speakers
for indexj in range(len(dataSet)):
	min1 = abs(relation_cut[indexj][0]-classification[0])
	min2 = abs(relation_cut[indexj][0]-classification[1])
	min3 = abs(relation_cut[indexj][0]-classification[2])
	min4 = abs(relation_cut[indexj][0]-classification[3])
	if min1<min2 and min1 < min3 and min1<min4:
		relation_cut_regonize.append([relation_cut[indexj],"Speaker1"])
	else:
		if min2<min1 and min2 < min3 and min2<min4:
			relation_cut_regonize.append([relation_cut[indexj],"Speaker2"])
		else:
			if min3<min1 and min3 < min2 and min3<min4:
				relation_cut_regonize.append([relation_cut[indexj],"Speaker3"])
			else:
				if min4<min1 and min4 < min2 and min4<min3:
					relation_cut_regonize.append([relation_cut[indexj],"Speaker4"])

- On afficher le result de reconnaissance avec une iterateur:

In [14]:
#step4: shows the result
print "=========The data after regonizing as 4 speakers==========="
for item in relation_cut_regonize:
	print item

=========The data after regonizing as 4 speakers===========
[[193.04016786970868, 0.1], 'Speaker4']
[[322.80179095213964, 0.15000000000000002], 'Speaker3']
[[48.618235376979669, 3.75], 'Speaker2']
[[206.06284495930311, 3.8000000000000003], 'Speaker4']
[[264.46484990545093, 5.800000000000001], 'Speaker3']
[[399.62976041628497, 6.7], 'Speaker3']
[[185.27945423237873, 6.75], 'Speaker4']
[[77.58484185258385, 11.05], 'Speaker2']
[[55.7106174875737, 12.700000000000001], 'Speaker2']
[[86.606563878601378, 13.100000000000001], 'Speaker2']
[[65.574334156685495, 16.6], 'Speaker2']
[[25.698883507655694, 18.75], 'Speaker1']
[[119.17666505648083, 18.85], 'Speaker2']
[[157.12783806256266, 18.900000000000002], 'Speaker4']
[[82.460244094641666, 24.5], 'Speaker2']
[[10.571385024471169, 25.150000000000002], 'Speaker1']
[[1.5434625659503354, 25.25], 'Speaker1']
[[211.66294698210436, 34.2], 'Speaker4']
[[70.795005105859445, 34.35], 'Speaker2']
[[100.16974802846812, 34.4], 'Speaker2']


### 3.  Conclusion

Dans ce mini-projet, nous avons appris à segmenter un fichier audio au format .wav. 

Pour la partie de segmentation. Nous avons réussi à couper l'audio. Cependant, nos résultats ne sont pas très précis, et nous soupçonnons cela est un problème de paramètre. Nous définissons le paramètre était trop sensible. Donc, nous avons mis beaucoup de coupe audio en petits morceaux.

Cependant, en regroupant la reconnaissance, nous avons finalement réussi à identifier les quatre personnes à l'intérieur de ce discours audio.